# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [120]:
#Load in data
aviation_df = pd.read_csv("data/AviationData.csv", encoding='ISO-8859-1', low_memory=False)

# Shape of the dataset
print("Dataset Shape (Rows, Columns):", aviation_df.shape)

# Preview the first few rows
print("/nFirst 5 rows of the dataset:")
print(aviation_df.head())

Dataset Shape (Rows, Columns): (88889, 31)
/nFirst 5 rows of the dataset:
         Event.Id Investigation.Type Accident.Number  Event.Date  \
0  20001218X45444           Accident      SEA87LA080  1948-10-24   
1  20001218X45447           Accident      LAX94LA336  1962-07-19   
2  20061025X01555           Accident      NYC07LA005  1974-08-30   
3  20001218X45448           Accident      LAX96LA321  1977-06-19   
4  20041105X01764           Accident      CHI79FA064  1979-08-02   

          Location        Country   Latitude   Longitude Airport.Code  \
0  MOOSE CREEK, ID  United States        NaN         NaN          NaN   
1   BRIDGEPORT, CA  United States        NaN         NaN          NaN   
2    Saltville, VA  United States  36.922223  -81.878056          NaN   
3       EUREKA, CA  United States        NaN         NaN          NaN   
4       Canton, OH  United States        NaN         NaN          NaN   

  Airport.Name  ... Purpose.of.flight Air.carrier Total.Fatal.Injuries  \
0   

In [121]:
# Check data types
print("\nData Types:")
print(aviation_df.dtypes)


Data Types:
Event.Id                   object
Investigation.Type         object
Accident.Number            object
Event.Date                 object
Location                   object
Country                    object
Latitude                   object
Longitude                  object
Airport.Code               object
Airport.Name               object
Injury.Severity            object
Aircraft.damage            object
Aircraft.Category          object
Registration.Number        object
Make                       object
Model                      object
Amateur.Built              object
Number.of.Engines         float64
Engine.Type                object
FAR.Description            object
Schedule                   object
Purpose.of.flight          object
Air.carrier                object
Total.Fatal.Injuries      float64
Total.Serious.Injuries    float64
Total.Minor.Injuries      float64
Total.Uninjured           float64
Weather.Condition          object
Broad.phase.of.flight      object
R

In [122]:
# Check for missing values (top 20)
print("\nTop 20 Columns with Most Missing Values:")
print(aviation_df.isnull().sum().sort_values(ascending=False).head(20))


Top 20 Columns with Most Missing Values:
Schedule                  76307
Air.carrier               72241
FAR.Description           56866
Aircraft.Category         56602
Longitude                 54516
Latitude                  54507
Airport.Code              38757
Airport.Name              36185
Broad.phase.of.flight     27165
Publication.Date          13771
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Fatal.Injuries      11401
Engine.Type                7096
Report.Status              6384
Purpose.of.flight          6192
Number.of.Engines          6084
Total.Uninjured            5912
Weather.Condition          4492
Aircraft.damage            3194
dtype: int64


In [123]:
# Summary statistics for numeric and object columns
print("\nSummary Statistics:")
print(aviation_df.describe(include='all'))


Summary Statistics:
              Event.Id Investigation.Type Accident.Number  Event.Date  \
count            88889              88889           88889       88889   
unique           87951                  2           88863       14782   
top     20001212X19172           Accident      CEN22LA149  1984-06-30   
freq                 3              85015               2          25   
mean               NaN                NaN             NaN         NaN   
std                NaN                NaN             NaN         NaN   
min                NaN                NaN             NaN         NaN   
25%                NaN                NaN             NaN         NaN   
50%                NaN                NaN             NaN         NaN   
75%                NaN                NaN             NaN         NaN   
max                NaN                NaN             NaN         NaN   

             Location        Country Latitude Longitude Airport.Code  \
count           88837         

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [124]:
#Data Cleaning – Filtering Aircrafts and Events

# Convert Event.Date to datetime
aviation_df["Event.Date"] = pd.to_datetime(aviation_df["Event.Date"], errors='coerce')

# Check how many invalid dates exist
print("Missing or malformed Event.Date values:", aviation_df["Event.Date"].isnull().sum())

# Drop rows where Event.Date is missing
aviation_df = aviation_df[aviation_df["Event.Date"].notnull()]

# Filter for events from 1983 onwards
aviation_df = aviation_df[aviation_df["Event.Date"].dt.year >= 1983]

# Check relevant columns
print("\nUnique Makes (sample):")
print(aviation_df["Make"].dropna().unique()[:10])

print("\nUnique Models (sample):")
print(aviation_df["Model"].dropna().unique()[:10])

Missing or malformed Event.Date values: 0

Unique Makes (sample):
['Piccard' 'Cessna' 'Piper' 'Balloon Works' 'North American' 'Beech'
 'Swearingen' 'Canadair' 'Javelin Aircraft' 'Douglas']

Unique Models (sample):
['AX-6' '182P' '182RG' 'PA-28R-200' '140' 'FIREFLY 7B' '340A' 'T-6G'
 'PA-24-250' 'PA-32-301R']


In [125]:
# Drop rows where Make or Model is missing (client requires real aircraft)
aviation_df = aviation_df[aviation_df["Make"].notnull() & aviation_df["Model"].notnull()]

# Report shape and missingness after filtering
print("\nShape after filtering:", aviation_df.shape)
print("\nRemaining missing values (top 15):")
print(aviation_df.isnull().sum().sort_values(ascending=False).head(15))


Shape after filtering: (85197, 31)

Remaining missing values (top 15):
Schedule                  73202
Air.carrier               68823
FAR.Description           56805
Aircraft.Category         56496
Longitude                 50862
Latitude                  50853
Airport.Code              36787
Airport.Name              34714
Broad.phase.of.flight     27092
Publication.Date          13763
Total.Serious.Injuries    12461
Total.Minor.Injuries      11886
Total.Fatal.Injuries      11361
Engine.Type                7024
Report.Status              6338
dtype: int64


### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [126]:
# Convert injury columns to numeric and fill NaNs with 0
injury_cols = [
    "Total.Fatal.Injuries", 
    "Total.Serious.Injuries", 
    "Total.Minor.Injuries", 
    "Total.Uninjured"
]

for col in injury_cols:
    aviation_df[col] = pd.to_numeric(aviation_df[col], errors='coerce').fillna(0)

# Create total number of people onboard
aviation_df["Total_Occupants"] = (
    aviation_df["Total.Fatal.Injuries"] + 
    aviation_df["Total.Serious.Injuries"] + 
    aviation_df["Total.Minor.Injuries"] + 
    aviation_df["Total.Uninjured"]
)

# Remove rows where total occupants is 0 (unrealistic for manned aircraft)
aviation_df = aviation_df[aviation_df["Total_Occupants"] > 0]

In [127]:
# Create injury severity metric: rate of fatal or serious injuries per occupant
aviation_df["SeriousOrFatal_Rate"] = (
    (aviation_df["Total.Fatal.Injuries"] + aviation_df["Total.Serious.Injuries"]) 
    / aviation_df["Total_Occupants"]
)

# Preview the new columns
print("\nSample of derived injury metrics:")
print(aviation_df[[
    "Make", "Model", 
    "Total.Fatal.Injuries", "Total.Serious.Injuries", 
    "Total_Occupants", "SeriousOrFatal_Rate"
]].head())


Sample of derived injury metrics:
         Make       Model  Total.Fatal.Injuries  Total.Serious.Injuries  \
3600  Piccard        AX-6                   0.0                     1.0   
3601   Cessna        182P                   0.0                     0.0   
3602   Cessna       182RG                   0.0                     0.0   
3603   Cessna        182P                   0.0                     0.0   
3604    Piper  PA-28R-200                   0.0                     0.0   

      Total_Occupants  SeriousOrFatal_Rate  
3600              2.0                  0.5  
3601              4.0                  0.0  
3602              2.0                  0.0  
3603              1.0                  0.0  
3604              2.0                  0.0  


**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [128]:
# Standardize Aircraft.Damage by converting to uppercase
aviation_df["Aircraft.damage"] = aviation_df["Aircraft.damage"].str.upper()

In [129]:
# Inspect unique values to understand what's present
print("Unique values in Aircraft.damage:")
print(aviation_df["Aircraft.damage"].value_counts(dropna=False))

Unique values in Aircraft.damage:
Aircraft.damage
SUBSTANTIAL    61473
DESTROYED      17460
NaN             2527
MINOR           2366
UNKNOWN           92
Name: count, dtype: int64


In [130]:
# Create a binary flag: 1 if Destroyed, 0 otherwise
aviation_df["Aircraft_Destroyed"] = aviation_df["Aircraft.damage"].apply(
    lambda x: 1 if x == "DESTROYED" else 0
)

In [131]:
# Show distribution
print("\nDistribution of Aircraft_Destroyed:")
print(aviation_df["Aircraft_Destroyed"].value_counts())


Distribution of Aircraft_Destroyed:
Aircraft_Destroyed
0    66458
1    17460
Name: count, dtype: int64


In [132]:
# Preview
print("\nSample rows with destruction indicator:")
print(aviation_df[["Make", "Model", "Aircraft.damage", "Aircraft_Destroyed"]].head())


Sample rows with destruction indicator:
         Make       Model Aircraft.damage  Aircraft_Destroyed
3600  Piccard        AX-6             NaN                   0
3601   Cessna        182P     SUBSTANTIAL                   0
3602   Cessna       182RG     SUBSTANTIAL                   0
3603   Cessna        182P     SUBSTANTIAL                   0
3604    Piper  PA-28R-200     SUBSTANTIAL                   0


### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [133]:
# Standardize casing and trim spaces
aviation_df["Make"] = aviation_df["Make"].str.upper().str.strip()

# Replace common variants
aviation_df["Make"] = aviation_df["Make"].replace({
    "BOEING COMPANY": "BOEING",
    "BOEING COMMERCIAL AIRPLANE CO.": "BOEING",
    "MCDONNELL DOUGLAS": "MCDONNELL-DOUGLAS",
    "MCDONNELL DOUGLAS CORPORATION": "MCDONNELL-DOUGLAS",
    "CESSNA AIRCRAFT": "CESSNA",
    "CESSNA AIRCRAFT CO": "CESSNA",
    "CESSNA AIRCRAFT COMPANY": "CESSNA",
    "PIPER AIRCRAFT": "PIPER",
    "PIPER AIRCRAFT INC.": "PIPER",
    "AIRBUS INDUSTRIE": "AIRBUS",
    "AIRBUS INDUSTRIES": "AIRBUS",
    "LOCKHEED AIRCRAFT CORPORATION": "LOCKHEED",
    "LOCKHEED MARTIN": "LOCKHEED",
    "BEECH AIRCRAFT": "BEECH",
    "BEECH AIRCRAFT CORPORATION": "BEECH",
    "ROBINSON HELICOPTER": "ROBINSON",
    "ROBINSON HELICOPTER CO.": "ROBINSON",
    "ROBINSON HELICOPTER COMPANY": "ROBINSON"
})

# Check value counts
make_counts = aviation_df["Make"].value_counts()
print("Make value counts (Top 15):")
print(make_counts.head(15))

# Apply threshold: keep only makes with >= 50 incidents
valid_makes = make_counts[make_counts >= 50].index
aviation_df = aviation_df[aviation_df["Make"].isin(valid_makes)]

# Final unique makes
print("\nNumber of makes retained:", len(valid_makes))
print("Sample of retained makes:", list(valid_makes[:10]))

Make value counts (Top 15):
Make
CESSNA               25753
PIPER                14106
BEECH                 5088
BELL                  2574
BOEING                2098
ROBINSON              1606
MOONEY                1275
GRUMMAN               1068
BELLANCA               982
HUGHES                 878
SCHWEIZER              753
AIR TRACTOR            668
AERONCA                606
MAULE                  571
MCDONNELL-DOUGLAS      562
Name: count, dtype: int64

Number of makes retained: 94
Sample of retained makes: ['CESSNA', 'PIPER', 'BEECH', 'BELL', 'BOEING', 'ROBINSON', 'MOONEY', 'GRUMMAN', 'BELLANCA', 'HUGHES']


### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [134]:
# Drop rows where Model is missing
aviation_df = aviation_df[aviation_df["Model"].notnull()]

# Check the number of unique models
num_unique_models = aviation_df["Model"].nunique()
print(f"Number of unique Model values: {num_unique_models}")

# Check the number of unique (Make, Model) combinations
num_unique_make_models = aviation_df[["Make", "Model"]].drop_duplicates().shape[0]
print(f"Number of unique (Make, Model) combinations: {num_unique_make_models}")

Number of unique Model values: 6146
Number of unique (Make, Model) combinations: 6732


In [135]:
# Investigate whether models are shared across multiple makes
model_to_make_counts = aviation_df.groupby("Model")["Make"].nunique().sort_values(ascending=False)
print("\nTop Models used by multiple Makes:")
print(model_to_make_counts[model_to_make_counts > 1].head(10))


Top Models used by multiple Makes:
Model
500       7
G-164A    7
G-164B    6
G-164C    5
AA-5B     5
AA-5      5
G-164     5
8KCAB     4
690A      4
690B      4
Name: Make, dtype: int64


In [136]:
# Create a unique aircraft identifier column
aviation_df["Aircraft_Type"] = aviation_df["Make"] + " " + aviation_df["Model"]

# Preview the result
print("\nSample Aircraft_Type values:")
print(aviation_df[["Make", "Model", "Aircraft_Type"]].head())


Sample Aircraft_Type values:
        Make       Model     Aircraft_Type
3601  CESSNA        182P       CESSNA 182P
3602  CESSNA       182RG      CESSNA 182RG
3603  CESSNA        182P       CESSNA 182P
3604   PIPER  PA-28R-200  PIPER PA-28R-200
3605  CESSNA         140        CESSNA 140


### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [137]:
# Clean Engine.Type
aviation_df["Engine.Type"] = aviation_df["Engine.Type"].str.upper().str.strip()
aviation_df["Engine.Type"] = aviation_df["Engine.Type"].replace({
    "NONE": "NONE",
    "UNK": "UNKNOWN"
})

# Clean Weather.Condition
aviation_df["Weather.Condition"] = aviation_df["Weather.Condition"].str.upper().str.strip()
aviation_df["Weather.Condition"] = aviation_df["Weather.Condition"].replace({
    "UNK": "UNKNOWN"
})

# Clean Purpose.of.flight
aviation_df["Purpose.of.flight"] = aviation_df["Purpose.of.flight"].str.title().str.strip()
aviation_df["Purpose.of.flight"] = aviation_df["Purpose.of.flight"].replace({
    "Air Race/Show": "Air Race/Show",
    "Air Race Show": "Air Race/Show",
    "Pubs": "Other/Unknown",
    "Asho": "Other/Unknown",
    "Publ": "Other/Unknown"
})

# Clean Broad.phase.of.flight
aviation_df["Broad.phase.of.flight"] = aviation_df["Broad.phase.of.flight"].str.title().str.strip()

# Preview cleaned categories
print("\nEngine.Type:", aviation_df["Engine.Type"].unique())
print("\nWeather.Condition:", aviation_df["Weather.Condition"].unique())
print("\nPurpose.of.flight (sample):", aviation_df["Purpose.of.flight"].value_counts().head())
print("\nBroad.phase.of.flight (sample):", aviation_df["Broad.phase.of.flight"].value_counts().head())  


Engine.Type: ['RECIPROCATING' 'UNKNOWN' 'TURBO PROP' 'TURBO FAN' 'TURBO SHAFT' nan
 'TURBO JET' 'GEARED TURBOFAN' 'LR' 'NONE']

Weather.Condition: ['VMC' 'IMC' 'UNKNOWN' nan]

Purpose.of.flight (sample): Purpose.of.flight
Personal              37457
Instructional          9439
Unknown                5475
Aerial Application     4144
Business               3449
Name: count, dtype: int64

Broad.phase.of.flight (sample): Broad.phase.of.flight
Landing        13104
Takeoff         9869
Cruise          8463
Maneuvering     6347
Approach        5255
Name: count, dtype: int64


### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [138]:
# Define threshold: drop columns where more than 50% of rows are missing
threshold = 0.5 * len(aviation_df)

# Count missing values in each column
missing_counts = aviation_df.isnull().sum()

# Identify columns that exceed the threshold
cols_to_drop = missing_counts[missing_counts > threshold].index.tolist()

# Drop these columns from the DataFrame
aviation_df.drop(columns=cols_to_drop, inplace=True)

# Output results
print("Dropped columns due to more than 50% missing values:")
print(cols_to_drop)

print("\nRemaining columns in the cleaned DataFrame:")
print(aviation_df.columns.tolist())

Dropped columns due to more than 50% missing values:
['Latitude', 'Longitude', 'Aircraft.Category', 'FAR.Description', 'Schedule', 'Air.carrier']

Remaining columns in the cleaned DataFrame:
['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date', 'Location', 'Country', 'Airport.Code', 'Airport.Name', 'Injury.Severity', 'Aircraft.damage', 'Registration.Number', 'Make', 'Model', 'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'Purpose.of.flight', 'Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status', 'Publication.Date', 'Total_Occupants', 'SeriousOrFatal_Rate', 'Aircraft_Destroyed', 'Aircraft_Type']


### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [139]:
aviation_df.to_csv("data/cleaned_aviation_data.csv", index=False)